In [130]:
import PIL 
import os, sys
import numpy as np
from collections import defaultdict as dic
from PIL import Image as im

fig = im.open('meteor_challenge_01.png')

# Parte simples achar por cor as informações pedidas
# Funcao que retorna duas informacoes, 
# numero de estrelas e numero de meteoro
# (Util para validacao de resultados)
def star_meteor(fig_, w_blue_=0):
    by_li = dic(int)
    for pix_ in fig_.getdata():
         by_li[pix_] += 1
    if w_blue_ == 0:
        return by_li[(255,255,255,255)], by_li[(255,0,0,255)]
    else:
        return by_li[(255,255,255,255)], by_li[(255,0,0,255)], by_li[(0,0,255,255)]

# Aplicando a funcao anterior 
star1, meteor1, water1 = star_meteor(fig,1)

# Branco possui todas as cores pelo espectro de cor
# entao sera #FFFFFF
print('Estrela = ', star1)

# Vermelho tem o maximo em vermelho e o resto zerado
# sendo entao #FF0000
print('Meteoro = ', meteor1)

# Azul tem o maximo em azul e o resto zerado sendo
# entao #0000FF
print('Water = ', water1)

w, h = fig.size
print(w,h)

# Transformando a figura em um array para poder
# encontrar os meteoros "marinhos"
mar = np.asarray(fig)

# Caso precise transformar em imagem
#<var_da_imagem> = im.fromarray(np.uint8(<var_do_array>))

# Função para reduzir array, tirando linhas sem importancia 
# como as apenas com ceu e deixando as outras
def reducerr(mar_):
    # Parte que retira o ceu sem estrela ou meteoro
    coun_ = []
    for i_ in range(len(mar_)):
        situ_ = 0
        temp_ = mar_[i_][0]

        for j_ in range(len(mar_[0])):
            if mar_[i_][j_,0] != temp_[0]:
                break
            if mar_[i_][j_,1] != temp_[1]:
                break
            if mar_[i_][j_,2] != temp_[2]:
                break
            if mar_[i_][j_,3] != temp_[3]:
                break
            if j_ == (len(mar_[0])-1):
                situ_ = 1
        if situ_ == 1:
            coun_.append(i_)
    for i_ in reversed(range(len(coun_))):
        deleter_ = coun_[i_]
        mar_ = np.delete(mar_,(deleter_), axis=0)
    
    # Parte que retira o solo pos primeira linha azul
    find_blue_ = []
    blue_ = (0,0,255,255)
    for i_ in range(len(mar_)):
        for j_ in range(len(mar_[0])):
            if np.all(blue_ == mar_[i_][j_]):
                find_blue_.append(i_)
                break
    find_blue_ = np.delete(find_blue_,(0), axis=0) # retirando a linha 1
    for i_ in reversed(range(len(find_blue_))):
        deleter_ = find_blue_[i_]
        mar_ = np.delete(mar_,(deleter_), axis=0)
    tot_ = len(mar_) - find_blue_[0]
    for i_ in reversed(range(1,tot_)):
        deleter_ = find_blue_[0] + i_
        mar_ = np.delete(mar_,(deleter_), axis=0)
    
    # Removendo o solo entre agua e primeira estrela
    last_white_or_red_ = 0
    white_ = (255,255,255,255)
    red_ = (255,0,0,255)
    for i_ in range(len(mar_)):
        for j_ in range(len(mar_[0])):
            if np.all(white_ == mar_[i_][j_]) or np.all(red_ == mar_[i_][j_]):
                last_white_or_red_ = i_
                break
    coun = len(mar_) - last_white_or_red_ - 2
    for i_ in reversed(range(1,coun)):
        deleter_ = last_white_or_red_ + i_
        mar_ = np.delete(mar_,(deleter_), axis=0)
    return mar_

red_arr = reducerr(mar)
red_fig = im.fromarray(np.uint8(red_arr))
   

star2, meteor2, water2 = star_meteor(red_fig,1)

# Branco possui todas as cores pelo espectro de cor
# entao sera #FFFFFF CONFERENCIA
print('Estrela = ', star2)

# Vermelho tem o maximo em vermelho e o resto zerado
# sendo entao #FF0000 CONFERENCIA
print('Meteoro = ', meteor2)

# Azul tem o maximo em azul e o resto zerado sendo
# entao #0000FF
print('Water = ', water2)

w, h = red_fig.size
print(w,h)

if  star2 != star1 or meteor2 != meteor1:
    print('Oh no') #fazer interrupção


# Cria uma imagem totalmente preta para que possa ser usada
def black_image_arr(ima_biar_):
    black_biar_ = ima_biar_.copy()
    black_biar_ = np.asarray(black_biar_)
    black_c_biar_ = np.uint8((0,0,0,255))
    if len(black_biar_) != 1:
        for i_biar_ in range(len(black_biar_)):
            for j_biar_ in range(len(black_biar_[0])):
                black_biar_[i_biar_][j_biar_] = black_c_biar_
    return black_biar_

def color_test(pos_test_,r=0,b=0,w=0):
    color_test_exit_ = 0
    if r == 1:
        red_ = np.uint8((255,0,0,255))
        if np.all(red_ == pos_test_):
            color_test_exit_ = 1
            #print(pos_test_)
    if b == 1:
        blue_ = np.uint8((0,0,255,255))
        if np.all(blue_ == pos_test_):
            color_test_exit_ = 2
            #print(pos_test_)
    if w == 1:
        white_ = np.uint8((255,255,255,255))
        if np.all(white_ == pos_test_):
            color_test_exit_ = 3
            #print(pos_test_)
    return color_test_exit_ 

def white_red_black(mar_wrb_):
    exit_arr_wrb_ = black_image_arr(mar_wrb_)
    for i_wrb_ in range(len(mar_wrb_)):
        for j_wrb_ in range(len(mar_wrb_[0])):
            white_ = (255,255,255,255)
            blue_ = (0,0,255,255)
            red_ = (255,0,0,255)
            if np.all(red_ == mar_wrb_[i_wrb_][j_wrb_]):
                exit_arr_wrb_[i_wrb_][j_wrb_] = mar_wrb_[i_wrb_][j_wrb_]
            if np.all(blue_ == mar_wrb_[i_wrb_][j_wrb_]):
                exit_arr_wrb_[i_wrb_][j_wrb_] = mar_wrb_[i_wrb_][j_wrb_]
            if np.all(white_ == mar_wrb_[i_wrb_][j_wrb_]):
                exit_arr_wrb_[i_wrb_][j_wrb_] = mar_wrb_[i_wrb_][j_wrb_]
    return exit_arr_wrb_

red_black_arr = white_red_black(red_arr)
red_black_fig = im.fromarray(np.uint8(red_black_arr))

# Teste para manter a consistencia
star3, meteor3, water3 = star_meteor(red_black_fig,1)

# Branco possui todas as cores pelo espectro de cor
# entao sera #FFFFFF CONFERENCIA
print('Estrela = ', star3)

# Vermelho tem o maximo em vermelho e o resto zerado
# sendo entao #FF0000 CONFERENCIA
print('Meteoro = ', meteor3)

# Azul tem o maximo em azul e o resto zerado sendo
# entao #0000FF
print('Water = ', water3)

w, h = red_black_fig.size
print(w,h)

by_li = dic(int)
for pix_ in red_black_fig.getdata():
     by_li[pix_] += 1
print(by_li)
red_black_fig


# "TRUNCAR" IMAGEM
def short_image(mar_si_):
    new_si_ = black_image_arr(mar_si_)
    new_si_ = np.delete(new_si_,(len(new_si_)-1), axis=0) # apaga ultima linha que esta preta
    height_arr_ = len(mar_si_)-2 # linha com azul
    new_si_[height_arr_] = mar_si_[height_arr_]
    height_arr_ -= 1
    width_arr_ = len(mar_si_[0])-1
    fig_temp_ver_ = im.fromarray(black_image_arr(mar_si_)).resize((1,height_arr_))
    fig_temp_hor_ = im.fromarray(black_image_arr(mar_si_)).resize((width_arr_,1))
    arr_temp_ver_ = np.asarray(fig_temp_ver_)
    arr_temp_hor_ = np.asarray(fig_temp_hor_)
    for j_si_ in range(len(mar_si_[0])):
        stor_si_ = []
        coun_si_ = 0
        temp_house_ver_ = arr_temp_ver_
        temp_house_hor_ = arr_temp_hor_
        for i_si_ in range(len(mar_si_)-1):
            if color_test(mar_si_[i_si_][j_si_],r=1,b=0,w=1) != 0:
                new_si_[coun_si_][j_si_] = mar_si_[i_si_][j_si_]
                coun_si_ += 1
            #if i_si_ == (len(mar_si_)-2):
                #print(new_si_[coun_si_-1][j_si_])
#        for k_si_ in range(coun_si_):
#            new_si_[k_si_][j_si_] = mar_si_[k_si_][j_si_]
 #           print(mar_si_[k_si_][j_si_],k_si_)
#        print(new_si_, j_si_)

        
        coun_si_ = 0
    return new_si_

all_tog_arr = short_image(red_black_arr)
all_tog_fig = im.fromarray(np.uint8(short_image(all_tog_arr)))

# Removendo o solo entre agua e primeira estrela
last_white_or_red_ = 0
white_ = (255,255,255,255)
red_ = (255,0,0,255)
for i_ in range(len(all_tog_arr)):
    for j_ in range(len(all_tog_arr[0])):
        if np.all(white_ == all_tog_arr[i_][j_]) or np.all(red_ == all_tog_arr[i_][j_]):
            last_white_or_red_ = i_
            break
coun = len(all_tog_arr) - last_white_or_red_ - 2
for i_ in reversed(range(1,coun)):
    deleter_ = last_white_or_red_ + i_
    all_tog_arr = np.delete(all_tog_arr,(deleter_), axis=0)

gif = im.fromarray(np.uint8(all_tog_arr))
gif.show()

print(star_meteor(gif,1))

gif.save('aliens_aquaticos.png','PNG',quality=100)

water_aliens = 0
for j_ in range(len(all_tog_arr[0])):
    summ = 0
    for i_ in range(len(all_tog_arr)):
        summ += color_test(all_tog_arr[i_][j_],r=1,b=0,w=0)
    if(color_test(all_tog_arr[i_][j_],r=0,b=1,w=0) == 2):
        water_aliens += summ
print('water_aliens', water_aliens)

# Parte das letras daqui para frente:


Estrela =  315
Meteoro =  328
Water =  1535
704 704
Estrela =  315
Meteoro =  328
Water =  221
704 366
Estrela =  315
Meteoro =  328
Water =  221
704 366
defaultdict(<class 'int'>, {(0, 0, 0, 255): 256800, (255, 255, 255, 255): 315, (255, 0, 0, 255): 328, (0, 0, 255, 255): 221})
(315, 328, 221)
water_aliens 105


In [2]:
print('numero de estrelas = ', star3)
print('numero de meteoros = ', meteor3)
print('numero de agua     = ', water3)
print('numero de meteoros na agua = ', water_aliens)


numero de estrelas =  315
numero de meteoros =  328
numero de agua     =  221
numero de meteoros na agua =  105


In [3]:
all_tog_arr

105


In [146]:
# all_tog_arr      e      gif = image

lin1_ = all_tog_arr
# Tira as duas ultimas linhas do array, 
# deixando apenas as dos meteoros e estrelas
espaco = 0
while len(lin1_) != 2:
    lin1_ = np.delete(lin1_,(2), axis=0)

# Retira o espaço extra e coloca nome como novo
novo = [[]]
novo.append([])
for i_ in range(len(lin1_[0])-1):
    if i_ == len(lin1_[0]):
        break
    if not(np.all(lin1_[0][i_] == (0,0,0,255)) and np.all(lin1_[0][i_+1] == (0,0,0,255))):
        novo[0].append(lin1_[0][i_])
        novo[1].append(lin1_[1][i_])

# Coloca a ultima linha preta para ser zero depois
if np.any(novo[0][len(novo[0])-1] != [0,0,0,255]):
    novo[0].append([0,0,0,255])
    novo[1].append([0,0,0,255])

# Conta o numero de pretos, para saber quantos espaços
novo2 = novo
#im.fromarray(np.uint8(novo)).show()
#print(novo[0][len(novo[0])-1])
dor = 0
preto = [0,0,0,255]
for i in range(len(novo[0])):
    if np.all(novo[0][i] == preto):
        dor += 1
#print(dor)


# Transforma preto [  0,  0,  0, 255] em 0 e
# [255,0,0,255] ou [255,255,255, 255] em 1
def testador(in_):
    # 0 é igual a preto
    # 1 é igual a branco
    out_ = 1
    if(np.all(in_ == [0,0,0,255])):
        out_ = 0
    return out_

# Faz 2 array com apenas zeros e uns
temp = [[]]
temp.append([])
for i_ in range(len(novo2[0])-1):
    temp[0].append(testador(novo2[0][i_]))
    temp[1].append(testador(novo2[1][i_]))
#print(temp)


# Tira se tiver mais que 6 posicoes brancas seguidas
# em codigo morse não existe 7 seguidos, se tiver
# precisa ser tirado
#def tira_branco()
posicoes = []
for i_tb_ in range(len(temp[0])-8):
    ok_ = 0
    for j_tb_ in range(7):
        if temp[0][i_tb_ + j_tb_] == 0:
            ok_ = 1
    if ok_ == 0:
        for j_tb_ in range(20):
            posicoes.append([i_tb_ + j_tb_])
            if temp[0][i_tb_ + j_tb_] == 0:
                break
    #




# Transforma coluna em 0 ou 1 dependendo da situação
def dot_tra(a_, b_, dt_=0):
    out_ = 0
    if a_ == 0:
        out_ = ' '
    else:
        if dt_ == 0:
            if b_ == 0:
                out_ = '.'
            else:
                out_ = '-'
        else:
            if b_ == 0:
                out_ = '-'
            else:
                out_ = '.'
    return out_

morse0_ = ''
morse1_ = ''
for i_ in range(len(temp[1])-2):
    morse0_ += dot_tra(temp[0][i_+1],temp[1][i_+1],0)
    morse1_ += dot_tra(temp[0][i_+1],temp[1][i_+1],1)
print(morse0_)

# Resultados que foram encontrados
message0 = ".-. -|--. . . .-. .-. --. ... - ... .- ... -. -...|.-. ... .- . -- ....|--. ..- --... - --.- . -- ---- --- -.- - -- . .. -- - --...- .- . .-. . --.. - -- -.-- --. - - - -- . . -- -... --. - - ... .- . .. -. .. - --. - . .-. ... .-. .. .- .. . -- ...- --. .- -- -..-|--. -.. - -- ... -- ---- --- .-. .-. ... -- . . --. .. --. .. .- .. - .-.. . .- --.- .-. . . ... . -- ..-- -- .- -- -.-. .-.. . .- ... -. ... --. . . .- .. -- . . -- -..- --. . . - .... -.. .-|-- .- .-. .. -- ...- --. ..- --. - . --. . . .-. . .- .. .|--- -. -- . - -- --. -- .-.. .- .... -- .-- -- .-"
message1 = ".-. - --. . . .-. .-. --. ... - ... .- ... -. -... .-. ... .- . -- .... --. ..- --... - --.- . -- ---- --- -.- - -- . .. -- - --...- .- . .-. . --.. - -- -.-- --. - - - -- . . -- -... --. - - ... .- . .. -. .. - --. - . .-. ... .-. .. .- .. . -- ...- --. .- -- -..- --. -.. - -- ... -- ---- --- .-. .-. ... -- . . --. .. --. .. .- .. - .-.. . .- --.- .-. . . ... . -- ..-- -- .- -- -.-. .-.. . .- ... -. ... --. . . .- .. -- . . -- -..- --. . . - .... -.. .- -- .- .-. .. -- ...- --. ..- --. - . --. . . .-. . .- .. . --- -. -- . - -- --. -- .-.. .- .... -- .-- -- .-"
m1 = 'RT#GEERRGSTSASNB#RSAEMH#GU7TQEMĤOKTMEIMT¡AEREZTMYGTTTMEEMBGTTSAEINITGTERSRIAIEMVGAMX#GDTMSMĤORRSMEEGIGIAITLEAQREESEMÜMAMCLEASNSGEEAIMEEMXGEETHDA#MARIMVGUGTEGEEREAIE#ONMETMGMLAHMWMA'
m2 = 'KE#UTTKKUOEONOAJ#KONTIĤ#UG2EFTIHSREITMIE#NTKTÜEILUEEEITTIJUEEONTMAMEUETKOKMNMTIÓUNIP#UWEIOIHSKKOITTUMUMNMEYTNFKTTOTIZINIÄYTNOAOUTTNMITTIPUTTEĤWN#INKMIÓUGUETUTTKTNMT#SAITEIUIYNĤIDIN'
message2 = "-.- .|..- - - -.- -.- ..- --- . --- -. --- .- .---|-.- --- -. - .. ----|..- --. ..--- . ..-. - .. .... ... .-. . .. - -- .. . ..---. -. - -.- - ..-- . .. .-.. ..- . . . .. - - .. .--- ..- . . --- -. - -- .- -- . ..- . - -.- --- -.- -- -. -- - .. ---. ..- -. .. .--.|..- .-- . .. --- .. .... ... -.- -.- --- .. - - ..- -- ..- -- -. -- . -.-- - -. ..-. -.- - - --- - .. --.. .. -. .. .-.- -.-- - -. --- .- --- ..- - - -. -- .. - - .. .--. ..- - - . ---- .-- -.|.. -. -.- -- .. ---. ..- --. ..- . - ..- - - -.- - -. -- -|... .- .. - . .. ..- .. -.-- -. ---- .. -.. .. -."
message3 = "-.- . ..- - - -.- -.- ..- --- . --- -. --- .- .--- -.- --- -. - .. ---- ..- --. ..--- . ..-. - .. .... ... .-. . .. - -- .. . ..---. -. - -.- - ..-- . .. .-.. ..- . . . .. - - .. .--- ..- . . --- -. - -- .- -- . ..- . - -.- --- -.- -- -. -- - .. ---. ..- -. .. .--. ..- .-- . .. --- .. .... ... -.- -.- --- .. - - ..- -- ..- -- -. -- . -.-- - -. ..-. -.- - - --- - .. --.. .. -. .. .-.- -.-- - -. --- .- --- ..- - - -. -- .. - - .. .--. ..- - - . ---- .-- -. .. -. -.- -- .. ---. ..- --. ..- . - ..- - - -.- - -. -- - ... .- .. - . .. ..- .. -.-- -. ---- .. -.. .. -."







# Teste VER
def cansei(mar_c_,dd_=0):
    gan_temp = ''
    for i_ in range(len(mar_c_[0])-1):
        if np.all(mar_c_[0][i_] == [0,0,0,255]):
            gan_temp += ' '
        elif dd_ == 1:
            if np.all(mar_c_[1][i_] == [0,0,0,255]):#dot
                gan_temp += '.'
            else:
                gan_temp += '-'
        else:
            if np.all(mar_c_[1][i_] == [0,0,0,255]):#dash
                gan_temp += '-'
            else:
                gan_temp += '.'
    return gan_temp

lin1_ = all_tog_arr
while len(lin1_) != 2:
    lin1_ = np.delete(lin1_,(2), axis=0)


aaaaa = cansei(lin1_,0)
print(' ')
bbbbb = cansei(lin1_,1)


for i_ in range(len(lin1_[0])-1):
    if i_ == len(lin1_[0]):
        break
    if not(np.all(lin1_[0][i_] == [0,0,0,255]) and np.all(lin1_[0][i_+1] == (0,0,0,255))):
        novo[0].append(lin1_[0][i_])
        novo[1].append(lin1_[1][i_])







#im.fromarray(np.uint8(short_image(red_black_arr)))
#a1 = short_image(red_black_arr)
#a.append(a1[len(red_black_arr)-2])
#for i in range(len(red_black_arr[0])):
#    print('new ',a[0][i],' old ',red_black_arr[len(red_black_arr)-2][i])
#temp_house_ =  np.asarray(im.new('RGBA',(1,len(new_si_))))
#print(type(temp_house_))
#im.fromarray(np.uint8(a1[len(a1)-1]))
#print(star1,meteor1,water1)
#print(star2,meteor2,water2)
#print(star3,meteor3,water3)
#print(star_meteor(im.fromarray(np.uint8(short_image(red_black_arr))),1))

#im.fromarray(np.uint8(short_image(red_black_arr))).show()

.-. - -.....- --. . . .-. .-. --. ... - ... .- ... -. -... --..-.. .-. ... .- . -- .... --..-.- --. ..- --... - --.- . -- ---- --- -.- - -- . .. -- - --...- .- . .-. . --.. - -- -.-- --. - - - -- . . -- -... --. - - ... .- . .. -. .. - --. - . .-. ... .-. .. .- .. . -- ...- --. .- -- -..- --..-.. --. -.. - -- ... -- ---- --- .-. .-. ... -- . . --. .. --. .. .- .. - .-.. . .- --.- .-. . . ... . -- ..-- -- .- -- -.-. .-.. . .- ... -. ... --. . . .- .. -- . . -- -..- --. . . - .... -.. .- --.-... -- .- .-. .. -- ...- --. ..- --. - . --. . . .-. . .- .. . -....-- --- -. -- . - -- --. -- .-.. .- .... -- .-- -- .-
 


In [61]:
print(type(color_test(red_black_arr[0][0][0],r=1,b=0,w=1)))
print(type(np.uint8((0,0,0,255))[0]))

<class 'int'>
<class 'numpy.uint8'>


In [6]:
color_test_exit_ = black_image_arr(red_black_arr)[0][0]
color_test_exit_

array([  0,   0,   0, 255], dtype=uint8)

In [7]:
'''
for i_ in range(len(lin1_[0])-1):
    if i_ == len(lin1_[0]):
        break
    if not(np.all(lin1_[0][i_] == (0,0,0,255)) and np.all(lin1_[0][i_+1] == (0,0,0,255))):
        novo[0].append(lin1_[0][i_])
        novo[1].append(lin1_[1][i_])
'''

[  0   0   0 255]


In [129]:
# Python program to implement Morse Code Translator
  
'''
VARIABLE KEY
'cipher' -> 'stores the morse translated form of the english string'
'decipher' -> 'stores the english translated form of the morse string'
'citext' -> 'stores morse code of a single character'
'i' -> 'keeps count of the spaces between morse characters'
'message' -> 'stores the string to be encoded or decoded'
'''
  
# Dictionary representing the morse code chart
MORSE_CODE_DICT = { 'A':'.-', 'B':'-...',
                    'C':'-.-.', 'D':'-..', 'E':'.',
                    'F':'..-.', 'G':'--.', 'H':'....',
                    'I':'..', 'J':'.---', 'K':'-.-',
                    'L':'.-..', 'M':'--', 'N':'-.',
                    'O':'---', 'P':'.--.', 'Q':'--.-',
                    'R':'.-.', 'S':'...', 'T':'-',
                    'U':'..-', 'V':'...-', 'W':'.--',
                    'X':'-..-', 'Y':'-.--', 'Z':'--..',
                    '1':'.----', '2':'..---', '3':'...--',
                    '4':'....-', '5':'.....', '6':'-....',
                    '7':'--...', '8':'---..', '9':'----.',
                    '0':'-----', ', ':'--..--', '.':'.-.-.-',
                    '?':'..--..', '/':'-..-.', '-':'-....-',
                    '(':'-.--.', ')':'-.--.-'}

# Function to encrypt the string
# according to the morse code chart
def encrypt(message):
    cipher = ''
    for letter in message:
        if letter != ' ':
  
            # Looks up the dictionary and adds the
            # correspponding morse code
            # along with a space to separate
            # morse codes for different characters
            cipher += MORSE_CODE_DICT[letter] + ' '
        else:
            # 1 space indicates different characters
            # and 2 indicates different words
            cipher += ' '
  
    return cipher
  
# Function to decrypt the string
# from morse to english
def decrypt(message):
    # extra space added at the end to access the
    # last morse code
    while message [0] == ' ':
        message = message[1:]
    cos2 = 0
    posis = 0
    for letter in message:
        print(letter)
        if letter != ' ':
            posis += 1
            cos2 += 1
        if cos2 > 6:
            while message[posis] != ' ':
                posis += 1
                cos2 += 1
            for i in range(0,cos2):
                message = message.replace(message[i],' ')
#            tempori = posis - cos2
 #           message = message[:tempori,pos:]
    
    message += ' '
  
    decipher = ''
    citext = ''
    for letter in message:
        # checks for space
        if (letter != ' '):
  
            # counter to keep track of space
            i_cm = 0
  
            # storing morse code of a single character
            citext += letter
  
        # in case of space
        else:
            # if i = 1 that indicates a new character
            i_cm += 1
  
            # if i = 2 that indicates a new word
            if i_cm == 2 :
  
                 # adding space to separate words
                decipher += ' '
            else:
  
                # accessing the keys using their values (reverse of encryption)
                decipher += list(MORSE_CODE_DICT.keys())[list(MORSE_CODE_DICT
                .values()).index(citext)]
                citext = ''
  
    return decipher
  
message = "GEEKS-FOR-GEEKS"
result = encrypt(message.upper())
print (result)

message = "--. . . -.- ... -....- ..-. --- .-. -....- --. . . -.- ... "
result = decrypt(message)
print (result)
  


--. . . -.- ... -....- ..-. --- .-. -....- --. . . -.- ... 
-
-
.
 
.
 
.
 
-
.
-
 
.
.
.
 
-
.
.
.
.
-
 
.
.
-
.
 
-
-
-
 
.
-
.
 
-
.
.
.
.
-
 
-
-
.
 
.
 
.
 
-
.
-
 
.
.
.
 


UnboundLocalError: local variable 'i_cm' referenced before assignment

In [133]:
print(decrypt(aaaaa))
print(' ')
print(decrypt(bbbbb))

-
.
-
 
 
.
 
 
.
-
-
-
-
-
.
 
.
.
-
 
-
 
-
 
-
.
-
 
-
.
-
 
.
.
-
 
-
-
-
 
 
.
 
-
-
-
 
 
-
.
 
-
-
-
 
 
.
-
 
.
-
-
-
 
.
.
-
-
.
-
-
 
-
.
-
 
-
-
-
 
-
.
 
 
 
 
-
 
.
.
 
-
-
-
-
 
.
.
-
-
.
-
.
 
.
.
-
 
-
-
.
 
.
.
-
-
-
 
 
 
 
.
 
 
 
 
 
 
.
.
-
.
 
 
-
 
.
.
 
.
.
.
.
 
.
.
.
 
.
-
.
 
 
.
 
 
 
 
 
 
.
.
 
-
 
-
-
 
.
.
 
 
 
 
.
 
.
.
-
-
-
.
 
 
-
.
 
 
-
 
 
 
-
.
-
 
-
 
 
 
.
.
-
-
 
 
.
 
.
.
 
.
-
.
.
 
.
.
-
 
.
 
.
 
 
.
 
 
 
 
 
 
.
.
 
-
 
 
-
 
.
.
 
.
-
-
-
 
.
.
-
 
.
 
 
 
 
.
 
-
-
-
 
 
-
.
 
-
 
-
-
 
.
-
 
-
-
 
.
 
.
.
-
 
.
 
-
 
-
.
-
 
-
-
-
 
-
.
-
 
 
-
-
 
-
.
 
-
-
 
-
 
.
.
 
-
-
-
.
 
.
.
-
 
-
.
 
 
.
.
 
.
-
-
.
 
.
.
-
-
.
-
-
 
.
.
-
 
.
-
-
 
 
.
 
 
 
 
 
 
.
.
 
-
-
-
 
 
.
.
 
.
.
.
.
 
.
.
.
 
-
.
-
 
-
.
-
 
-
-
-
 
.
.
 
-
 
 
-
 
.
.
-
 
 
-
-
 
.
.
-
 
-
-
 
 
-
.
 
 
-
-
 
 
 
.
 
 
 
 
 
 
-
.
-
-
 
 
-
 
-
.
 
.
.
-
.
 
-
.
-
 
-
 
-
 
-
-
-
 
 
-
 
 
.
.
 
-
-
.
.
 
.
.
 
 
 
-
.
 
.
.
 
.
-
.
-
 
-
.
-
-
 
 
-
 
-
.
 
 


UnboundLocalError: local variable 'i_cm' referenced before assignment

In [132]:
print(aaaaa)

 -.-  .  .-----. ..- - - -.- -.- ..- ---  . ---  -. ---  .- .--- ..--.-- -.- --- -.    - .. ---- ..--.-. ..- --. ..---    .      ..-.  - .. .... ... .-.  .      .. - -- ..    . ..---.  -.  -   -.- -   ..--  . .. .-.. ..- . .  .      .. -  - .. .--- ..- .    . ---  -. - -- .- -- . ..- . - -.- --- -.-  -- -. -- - .. ---. ..- -.  .. .--. ..--.-- ..- .--  .      .. ---  .. .... ... -.- -.- --- .. -  - ..-  -- ..- --  -.  --   .      -.--  - -. ..-. -.- - - ---  -  .. --.. ..   -. .. .-.- -.--  - -.  --- .-  --- ..- - - -. --   .. - -  .. .--. ..- - -  . ---- .--  -. ..-.--- .. -.   -.- --  .. ---. ..- --. ..- . - ..- - - -.-  -  -. -- - .----.. ... .-  .. -  . .. ..-  .. -.-- -. ---- .. -..  .. -.-
